In [4]:
import tushare as ts
import numpy as np
ts.set_token("8c6e5026fd3e2b5f96f15553d72f84eaaea8182a9157b783a41b1be0")
pro = ts.pro_api()

### 获取当天所有可转债信息

In [60]:
# from tqdm import tqdm
# import gc
# gc.collect()
# all_codes = set()
# for date in tqdm(range(20180101,20210601)):
# #     print(date)
#     try: 
#         codes = pro.cb_daily(trade_date=date).ts_code
#         all_codes.update(codes)
#     except: 
#         pass  
# print(all_codes)   
# # 获取转债代码,前一天收盘价,交易日期，转债代码'ts_code','trade_date','pre_close'
# conv_daily = conv_daily.ts_code

In [61]:
# len(all_codes)

In [26]:
# import numpy as np   # 存储所有转债的代码
# all_codes_20180101_20210601 = np.array(list(all_codes))
# np.save('all_codes_20180101_20210601.npy',all_codes_20180101_20210601)

### 记录每只股票转股价变化表

In [62]:
# import os
# os.chdir(r'E:\quantTradeLeanner\convert_table')
# os.getcwd()
# import time # 每分钟最多访问200次
# for code in tqdm(all_codes):
#     df = pro.cb_price_chg(ts_code=code,fields="change_date,convert_price_initial,convertprice_aft")
#     df.to_csv(code.split('.')[0]+'_'+name.split('.')[1]+'.csv')
#     time.sleep(0.1)
#     del df

In [107]:
### 加载所有转债的代码
def get_all_codes():
    '''
        return: 加载所有转债的代码
    '''
    os.chdir(r'E:\quantTradeLeanner')
    all_codes = np.load('all_codes_20180101_20210601.npy')
    return all_codes

### 获取债股票代码转换字典
def get_d2s_dict(all_codes):
    '''
        all_codes: 这两年所有的债券代码
        return: 这两年所有的债券转股票代码
    '''
    df = pro.cb_basic(fields="ts_code,stk_code")
    df = df[df.ts_code.isin(all_codes)]
    df.reset_index()
    det2share_dict = df.set_index('ts_code').T.to_dict('list')
    return det2share_dict

### 查询债券价格
def get_daily_table(time):
    '''
        time: 时间
        return : 两列df, 债券代码和债券价格
    '''
    daily_table = pro.cb_daily(trade_date=time,fields='ts_code,pre_close')
    daily_table.columns = ['ts_code','pre_det_price']
    return daily_table
get_daily_table(time)

# 查询股票价格
def get_share_price(codes,time,det2share_dict):
    '''
        code: 当日代码列表
        time: 检索时间
        det2share_dict: 债券对应的正股代码字典
        return : pd.DataFrame 当日的正股价格序列
        作用: 检索当日可转债对应的正股价
    '''
    share_price = []
    for code in tqdm(codes):
        share_code = det2share_dict[code][0]
        share_price.append(pro.daily(ts_code=share_code, start_date=time, end_date=time).pre_close[0])
    share_price = pd.DataFrame(share_price,columns=['pre_share_price'])
    return  share_price

# 查询转股价格
def get_convprice_per_det(codes,time):
    '''
        code: 当日代码列表
        time: 检索时间
        return : pd.DataFrame 当日的转股价序列
        作用: 检索当日可转债对应的转股价
    '''
    os.chdir(r'E:\quantTradeLeanner\convert_table')
    conv_price_list = []
    for code in tqdm(codes):
        code_info = pd.read_csv(code.split('.')[0]+'_'+name.split('.')[1]+'.csv')
        date_list = list(code_info.change_date)
        price_list = list(code_info.convertprice_aft)
        price_list[0]=code_info.convert_price_initial[0]
        flag = 0
        i = 0
        for d in date_list:
            if int(time) > int(d):
                i += 1
            else:
                flag = i
        if flag == 0 or flag ==1:
            conv_price = price_list[0]
        else:
            conv_price = price_list[flag-1]
#             break
        conv_price_list.append(conv_price)
    conv_price_list = pd.DataFrame(conv_price_list,columns=['conv_price'])
    return conv_price_list
def get_final_table(daily_table,share_price,conv_price):
    '''
        return: 最终表格,包括溢价率
    '''
    df = pd.concat([daily_table,share_price,conv_price],axis=1)
    mid_value = (100 / df.conv_price) * df.pre_share_price
    over_rate = (df.pre_det_price-mid_value)/mid_value
    over_rate = pd.DataFrame(over_rate,columns=['over_rate'])
    df1 = pd.concat([df,over_rate],axis=1)
    return df1


# 获取20180202当天的溢价率

In [105]:
import numpy as np 
import os 
from tqdm import tqdm
import pandas as pd


all_codes = get_all_codes()
det2share_dict = get_d2s_dict(all_codes)

time = '20180102'
daily_table = get_daily_table(time)
share_price = get_share_price(daily_table.ts_code,time,det2share_dict)
conv_price = get_convprice_per_det(daily_table.ts_code,time)
get_final_table(daily_table,share_price,conv_price)

100%|█████████████████████████████████████████████████████████████████████████████████| 37/37 [00:00<00:00, 395.66it/s]


,ts_code,pre_det_price,pre_share_price,conv_price,over_rate
0,110030.SH,105.280,5.68,7.24,0.341949
1,113008.SH,99.510,6.69,10.37,0.542479
2,110031.SH,99.900,21.54,42.80,0.985014
3,123001.SZ,96.600,5.52,9.77,0.709750
4,110032.SH,121.710,9.07,7.43,-0.002971
5,110033.SH,115.920,10.10,8.81,0.011144
6,110034.SH,109.620,18.95,18.52,0.071326
7,113009.SH,116.980,24.66,21.24,0.007565
8,128010.SZ,117.531,10.25,9.32,0.068672
9,113010.SH,104.130,5.64,9.30,0.717037
